In [1]:
import requests, bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [2]:
def parse_comp():
    #parse competition site
    url = 'https://fbref.com/en/comps/'
    response = requests.get(url)
    status = response.status_code
    if status == 200:
        page = response.text
        souppg = bs(page)
    else:
        print(f"Oops! Received status code {status}")  
    return souppg
    
    
def info_league(url,year_lst,league):
    #parse league history page
    response = requests.get(url)
    status = response.status_code
    if status == 200:
        page = response.text
        souppg = bs(page)
    else:
        print(f"Oops! Received status code {status}")
    
    df = pd.DataFrame({})
    for year in year_lst:
        #find link for each season
        if souppg.find('div',id = 'div_seasons').find(text = str(year) + '-' + str(year+1)) is not None:
            season_position = souppg.find('div',id = 'div_seasons').\
                find(text = str(year) + '-' + str(year+1)).parent
        else:
            season_position = souppg.find('div',id = 'all_seasons').find(text = str(year)).parent
            
        url = website + season_position['href']
        
        #find number of teams in each season
        num_squad = int(season_position.parent.parent.find('td',attrs={'data-stat':'num_squads'}).text)
        print(url,num_squad)
        
        # info_season carries the dataframe for each season
        df = pd.concat([df,info_season(num_squad,url)])
        df['year'] = year
        df['league'] = league
        
    return df
        
def info_season(num_squad,url):

    response = requests.get(url)
    status = response.status_code
    if status == 200:
        page = response.text
        souppg = bs(page)
    else:
        print(f"Oops! Received status code {status}")

    st_stats = souppg.find_all("div", {"id": "switcher_stats_squads_standard"})[0].find('tbody')
    
    # create the list of team in each season
    squadname = []
    for i in range(num_squad):
        row = st_stats.find_all('tr')[i]
        squadname.append(row.find('th',attrs={'data-stat':'squad'}).text)

    return pd.DataFrame(info_team(squadname,souppg)).set_index('squad')        

def info_team(squadname_lst,souppg):
    squadinfo = []
    for squadname in squadname_lst:
        # create dictionary with team name
        squad_dict = {'squad':squadname}
        for key, value in columns_dict.items():
            # find team name position in each table
            for item in souppg.find_all("div", {"id": key})[0].\
                    find('tbody').find_all('th',attrs={'data-stat':'squad'}):
                if item.find(text=re.compile(squadname)) is not None:
                    #find values of each column, if not integer, then turn to float
                    for column in value:
                        temp = item.parent.find('td',attrs={'data-stat':column}).text
                        if temp.isdigit():
                            squad_dict[column] = int(temp)
                        else:
                            squad_dict[column] = float(temp)
                            
        squadinfo.append(squad_dict)
    return squadinfo


In [3]:
global columns_dict, website

# key -- html table name; value -- html table column name
columns_dict = {'switcher_stats_squads_standard':['possession','games','goals'],
 'switcher_stats_squads_keeper':['goals_against_gk'],
'switcher_stats_squads_shooting':['shots_total','shots_on_target_pct'],
 'switcher_stats_squads_passing':['passes_pct_short','passes_pct_medium','passes_pct_long'],
 'switcher_stats_squads_defense':['tackles','tackles_won','dribble_tackles_pct','pressure_regain_pct','blocks','interceptions'],
 'switcher_stats_squads_possession':['touches','dribbles_completed_pct','carries','passes_received_pct']
}

website = 'https://fbref.com'

year_lst = range(2017,2022)

league_lst = ['Premier League','La Liga','Fußball-Bundesliga','Serie A','Ligue 1']

souppg = parse_comp()

df = pd.DataFrame({})

for league in league_lst:
    url = website + souppg.find(text=league).parent['href']
    df = pd.concat([df,info_league(url,year_lst,league)])    

league = 'Major League Soccer'
year_lst = range(2018,2022)
url = website + souppg.find(text=league).parent['href']
df = pd.concat([df,info_league(url,year_lst,league)])

df.head()

https://fbref.com/en/comps/9/1631/2017-2018-Premier-League-Stats 20
https://fbref.com/en/comps/9/1889/2018-2019-Premier-League-Stats 20
https://fbref.com/en/comps/9/3232/2019-2020-Premier-League-Stats 20
https://fbref.com/en/comps/9/10728/2020-2021-Premier-League-Stats 20
https://fbref.com/en/comps/9/Premier-League-Stats 20
https://fbref.com/en/comps/12/1652/2017-2018-La-Liga-Stats 20
https://fbref.com/en/comps/12/1886/2018-2019-La-Liga-Stats 20
https://fbref.com/en/comps/12/3239/2019-2020-La-Liga-Stats 20
https://fbref.com/en/comps/12/10731/2020-2021-La-Liga-Stats 20
https://fbref.com/en/comps/12/La-Liga-Stats 20
https://fbref.com/en/comps/20/1634/2017-2018-Bundesliga-Stats 18
https://fbref.com/en/comps/20/2109/2018-2019-Bundesliga-Stats 18
https://fbref.com/en/comps/20/3248/2019-2020-Bundesliga-Stats 18
https://fbref.com/en/comps/20/10737/2020-2021-Bundesliga-Stats 18
https://fbref.com/en/comps/20/Bundesliga-Stats 18
https://fbref.com/en/comps/11/1640/2017-2018-Serie-A-Stats 20
https

,possession,games,goals,goals_against_gk,shots_total,shots_on_target_pct,passes_pct_short,passes_pct_medium,passes_pct_long,tackles,...,dribble_tackles_pct,pressure_regain_pct,blocks,interceptions,touches,dribbles_completed_pct,carries,passes_received_pct,year,league
squad,,,,,,,,,,,,,,,,,,,,,
Arsenal,61.5,38,73,51,590,38.5,89.2,88.9,66.6,690,...,39.3,32.2,509,484,29511,61.0,19525,87.8,2021,Premier League
Bournemouth,47.8,38,45,61,473,32.3,86.3,83.2,54.2,582,...,34.7,27.5,622,466,22844,55.7,13059,83.0,2021,Premier League
Brighton,43.7,38,33,54,381,29.7,84.4,81.8,55.7,692,...,31.6,28.2,639,556,20800,62.1,11626,79.9,2021,Premier League
Burnley,43.9,38,35,39,388,32.5,83.5,79.2,47.9,578,...,34.8,27.5,675,529,19878,58.3,10366,77.3,2021,Premier League
Chelsea,55.4,38,60,38,602,34.7,88.8,88.7,65.9,720,...,40.8,32.8,510,514,27447,60.4,18152,86.6,2021,Premier League


In [5]:
df.to_csv('soccer_statistics.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 590 entries, Arsenal to Vancouver
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   possession              590 non-null    float64
 1   games                   590 non-null    int64  
 2   goals                   590 non-null    int64  
 3   goals_against_gk        590 non-null    int64  
 4   shots_total             590 non-null    int64  
 5   shots_on_target_pct     590 non-null    float64
 6   passes_pct_short        590 non-null    float64
 7   passes_pct_medium       590 non-null    float64
 8   passes_pct_long         590 non-null    float64
 9   tackles                 590 non-null    int64  
 10  tackles_won             590 non-null    int64  
 11  dribble_tackles_pct     590 non-null    float64
 12  pressure_regain_pct     590 non-null    float64
 13  blocks                  590 non-null    int64  
 14  interceptions           590 non-nul